<a href="https://colab.research.google.com/github/josee2202/ETL_PROYECTO/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

# **Extracción:**

In [77]:
# Ruta del archivo CSV
# ==============================================================================
ruta_archivo_meets_csv = 'meets.csv'

# Extracción del archivo CSV
# ==============================================================================
try:
    # Leer el archivo CSV con pandas
    df_meets = pd.read_csv(ruta_archivo_meets_csv)

    # Verificar la cantidad de filas importadas
    total_esperado = len(df_meets)
    total_real = 8482

    # Verificar si la cantidad de filas es correcta
    if total_esperado == total_real:
        resultado = f"Extracción del archivo '{ruta_archivo_meets_csv}' exitosa"
    else:
        resultado = 'Error: las filas importadas no coinciden'

    print(f"Resultado: {resultado}")

except Exception as e:
    print("Ocurrió un error durante la ejecución del ETL:", e)


Resultado: Extracción del archivo 'meets.csv' exitosa


In [78]:
# Ruta del archivo CSV
# ==============================================================================
ruta_archivo_open_csv = 'openpowerlifting.csv'


# Extracción del archivo CSV
# ==============================================================================
try:
    # Leer el archivo CSV con pandas
    df_open = pd.read_csv(ruta_archivo_open_csv)

    # Verificar la cantidad de filas importadas
    total_esperado = len(df_open)
    total_real = 386414

    # Verificar si la cantidad de filas es correcta
    if total_esperado == total_real:
        resultado = f"Extracción del archivo '{ruta_archivo_open_csv}' exitosa"
    else:
        resultado = 'Error: las filas importadas no coinciden'

    print(f"Resultado: {resultado}")

except Exception as e:
    print("Ocurrió un error durante la ejecución del ETL:", e)

Resultado: Extracción del archivo 'openpowerlifting.csv' exitosa


# **Transformación:**
## Calidad de los datos:

### Archivo 'meets.csv':
Antes de comenzar con el análisis de las métricas, dedicaremos un momento a revisar los datos del archivo 'meets.csv' y comprender su estructura. En este proyecto nos enfocaremos en diez métricas clave de calidad de datos.

In [ ]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
df_meets.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8482 entries, 0 to 8481
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MeetID       8482 non-null   int64 
 1   MeetPath     8482 non-null   object
 2   Federation   8482 non-null   object
 3   Date         8482 non-null   object
 4   MeetCountry  8482 non-null   object
 5   MeetState    5496 non-null   object
 6   MeetTown     6973 non-null   object
 7   MeetName     8482 non-null   object
dtypes: int64(1), object(7)
memory usage: 530.2+ KB


In [ ]:
# Cabezera del archivo
# ==============================================================================
df_meets.head(5)

,MeetID,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,0,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...
1,1,365strong/1602,365Strong,2016-11-19,USA,MO,Ozark,Thanksgiving Powerlifting Classic
2,2,365strong/1603,365Strong,2016-07-09,USA,NC,Charlotte,Charlotte Europa Games
3,3,365strong/1604,365Strong,2016-06-11,USA,SC,Rock Hill,Carolina Cup Push Pull Challenge
4,4,365strong/1605,365Strong,2016-04-10,USA,SC,Rock Hill,Eastern USA Challenge


In [ ]:
# Número de datos ausentes por columna
# ==============================================================================
df_meets.isna().sum().sort_values()

,0
MeetID,0
MeetPath,0
Federation,0
Date,0
MeetCountry,0
MeetName,0
MeetTown,1509
MeetState,2986


#### Precisión:
La precisión de los datos se evalúa contrastándolos con una fuente confiable de información. En nuestro caso, esto implica verificar los valores con la página web oficial de donde fueron obtenidos, Open Powerlifting (y no Kaggle). Al hacerlo, observamos que los valores faltantes se deben a la falta de información proporcionada por la web, y no a errores en la extracción de los datos. Por ello, consideramos que la precisión es completa en todas las columnas y podemos afirmar que la fuente es confiable, reflejando con exactitud los datos en el momento de la extracción.

Además, queremos asegurarnos de que la columna MeetID sea precisa, verificando que sus valores aumenten de uno en uno de forma correcta.

In [ ]:
# Función para verificar si MeetID aumenta en incrementos de uno
# ==============================================================================

def verificar_incremento_uno(df, columna):
    ids_ordenados = df[columna].sort_values().reset_index(drop=True)
    incremento_correcto = True

    for i in range(1, len(ids_ordenados)):
        if ids_ordenados[i] != ids_ordenados[i - 1] + 1:
            incremento_correcto = False
            print(f"Error en el incremento entre el valor {ids_ordenados[i-1]} y {ids_ordenados[i]}")

    # Resultado final
    if incremento_correcto:
        print("Los valores de MeetID aumentan en incrementos de 1.")
    else:
        print("Los valores de MeetID NO aumentan en incrementos de 1.")



In [ ]:
# Verificar si MeetID aumenta en incrementos de uno
# ==============================================================================
verificar_incremento_uno(df_meets, 'MeetID')

Los valores de MeetID aumentan en incrementos de 1.


#### Linaje:
El linaje evalúa el origen de los datos y su confiabilidad. En este caso, el linaje es sólido, ya que la fuente es la página oficial de Open Powerlifting, que tiene acceso directo a los datos de los atletas y competiciones. Además, es una de las principales fuentes de estadísticas de powerlifting a nivel mundial.

#### Semántica:
La semántica se refiere al significado correcto de los datos. Es decir, los datos deben tener un sentido claro y coherente. En este archivo, todas las columnas están bien definidas y no hay ambigüedades, por lo que la semántica de ambos archivos está completamente asegurada.

#### Estructura:
El análisis estructural se centra en verificar que los valores de los datos sigan un formato y patrón válido. En este caso, debemos asegurarnos de que la columna MeetPath tenga la estructura texto + barra + texto, ya que cualquier otra forma podría generar errores al intentar acceder a la URL. También es importante confirmar que la columna Fecha tenga el formato correcto YYYY-MM-DD. En cuanto a las demás columnas, consideramos que tienen una estructura adecuada, ya que, al tratarse de nombres de países, ciudades y estados, no se han encontrado errores en esos datos.

In [ ]:
# Revisar la estructura en la columna MeetPath
# ==============================================================================
incorrect_format_count = 0

for idx, row in df_meets.iterrows():
    meet_path = row['MeetPath']

    # Verificar que MeetPath tenga el formato: texto + barra + texto
    if not re.match(r'^[a-zA-Z0-9-]+/[a-zA-Z0-9-]+$', meet_path):
        incorrect_format_count += 1

print(f"Total de formatos incorrectos en MeetPath: {incorrect_format_count}")

Total de formatos incorrectos en MeetPath: 5


In [ ]:
# Revisar la estructura en la columna fecha
# ==============================================================================
all_good = True
for date in df_meets['Date']:
    try:
        pd.to_datetime(date, format='%Y-%m-%d')
    except ValueError:
        print(f"Fecha con formato incorrecto: {date}")
        all_good = False

if all_good:
    print("Todas las fechas están en el formato correcto (YYYY-MM-DD).")

Todas las fechas están en el formato correcto (YYYY-MM-DD).


In [ ]:
# Revisar la estructura en la columna de la federación
# ==============================================================================
print(*df_meets['Federation'].unique(), sep=', ')

365Strong, AAU, ADFPA, APA, APC, APF, WPC, AAPF, CAPO, AsianPF, BB, BP, BPU, CommonwealthPF, CPF, CPL, CPU, EPA, EPF, FESUPO, FFForce, FPO, GPA, GPC, GPC-AUS, GPC-GB, HERC, IPA, IPF, IrishPF, MHP, MM, NAPF, NASA, NIPF, NPA, NSF, NZPF, OceaniaPF, PA, ProRaw, RAW, RPS, XPC, RUPC, ScottishPL, SCT, SPF, THSPA, UPA, USAPL, USPF, USPA, IPL, WelshPA, WNPF, WRPF, WRPF-AUS, WRPF-CAN, WUAP


In [ ]:
# Revisar la estructura en la columna de la federación
# ==============================================================================
print(*df_meets['MeetCountry'].unique(), sep=', ')

USA, Australia, Finland, Ireland, Indonesia, Russia, UK, England, New Zealand, India, Canada, Italy, Czechia, Germany, Estonia, Spain, Denmark, Brazil, Peru, France, Malaysia, Serbia, Japan, Argentina, South Africa, Puerto Rico, Sweden, Norway, Colombia, Luxembourg, Poland, Belarus, Iceland, Netherlands, Cayman Islands, US Virgin Islands, N.Ireland, Israel, Singapore, New Caledonia, Ukraine, Greece, Scotland, Wales, Kazakhstan


In [ ]:
# Revisar la estructura en la columna de estado
# ==============================================================================
print(*df_meets['MeetState'].unique(), sep=', ')

NC, MO, SC, GA, IN, CA, NV, PA, FL, nan, OR, AK, MI, AL, TX, OH, AR, NJ, MS, VT, UT, ME, ND, IL, NSW, SD, LA, IA, ID, MN, SA, QLD, VIC, WA, MH, BC, ON, QC, NS, AB, SK, MB, NL, PE, NB, ACT, NY, CT, MD, WV, TN, CO, VA, WB, KS, OK, KY, NM, AZ, WI, WKO, WGN, NTL, AKL, OTA, CAN, BOP, STL, MWT, TAS, NT, MA, RI, NH, MT, NE, DC, DE, HI, WY, SP


In [ ]:
# Revisar la estructura en la columna de la ciudad
# ==============================================================================
print(*df_meets['MeetTown'].unique(), sep=', ')

Charlotte, Ozark, Rock Hill, Martinez, Lake Ozark, Fayetteville, Evansville, Conway, San Jose, Las Vegas, San Diego, Los Angeles, Santa Barbara, Santa Clara, Wilkes-Barre, nan, Portland, Clackamas, Searcy, Metamora, Tuscaloosa, Ellisville, El Paso, Lebanon, Newport, Hillsborough, Harrison, Largo, Philadelphia, Brattleboro, McAllen, Leadington, Jackson, St. George, Weslaco, Clawson, Park Hills, St. John's, Gainesville, Orlando, Grand Rapids, Westbrook, Fargo, Chicago, Jacksonville, Cronulla, Lombard, Chamberlain, Helsinki, Fresno, Baton Rouge, New Port Richey, Tampa, Juva, Lake Buena Vista, Santa Maria, Limerick, Ft. Jackson, Moss Point, Bandung, Moscow, Dover, Horncastle, London, Glasgow, Ashington, Northamptonshire, Gosport, Macclesfield, Perth, Northumberland, Christchurch, Pune, Bournemouth, Auckland, Richmond, Toronto, Waterloo, Longueuil, Amherst, Scarborough, Digby, Mississauga, Edmonton, Prince George, Mississaugua, Kitchener, Orleans, Burlington, Guelph, Barrie, Hamilton, Vaugh

In [ ]:
# Revisar la estructura en la columna del nombre de la competición
# ==============================================================================
print(*df_meets['MeetName'].unique(), sep=', ')

2016 Junior & Senior National Powerlifting Championships, Thanksgiving Powerlifting Classic, Charlotte Europa Games, Carolina Cup Push Pull Challenge, Eastern USA Challenge, New Year Challenge, Midwest Regionals, Victory Cup Challenge, Eastern Regional Championship, Ohio Valley Regional Open Powerlifting Championships, US Masters Open, 2017 Men & Women National Powerlifting Championships, East Coast Conflict, Kindred NorCal Powerlifting Championships, 2016 AAU Worlds, So Cal Meet, Battle of Los Angeles, Central California Meet, American Meet, Sin City Powerlifting, Men's Nationals, Ironfest Challenge, Raw Nationals, Elite Fall Classic, APA Winter Classic, Elite Spring Challenge District 7 Regionals, Rhino Open Championships, Michigan Summer Iron Bash, Raw Wars III, Missouri Raw Summer Iron Bash, Lone Star Raw Championships, Southern States, Garden State Raw Championships, Clash of the Iron Warriors, Blessed Iron Bash, Mississippi State Championships, 30th Can-Am International Champions

#### Completitud:
La completitud define el grado en que se rellenan los valores de datos necesarios y no se dejan en blanco. Vemos que casi todas las columnas están completas al 100%. Las únicas excepciones son las columnas del estado y ciudad de las competiciones.

In [ ]:
# Porcentaje de datos ausentes por columna
# ==============================================================================
(100 - (df_meets.isna().sum() * 100 / len(df_meets))).sort_values().round(2)

,0
MeetState,64.80
MeetTown,82.21
MeetID,100.00
MeetPath,100.00
Federation,100.00
Date,100.00
MeetCountry,100.00
MeetName,100.00


#### Consistencia:
La consistencia se asegura de que los datos no tengan contradicciones dentro del mismo registro, es decir, que sean consistentes en su significado, formato y estructura, incluso si provienen de diferentes fuentes. En nuestro caso, los registros se almacenan en un solo archivo, por lo que no hay con qué compararlos. Así que podemos decir que la consistencia es total.

#### Puntualidad:
La puntualidad se refiere al tiempo que tarda en accederse a la información solicitada. En este proyecto, descargamos los datasets desde Kaggle, que son archivos de tamaño manejable. No hemos tenido problemas ni al acceder a ellos ni al procesarlos. Por eso, podemos decir que la puntualidad es del 100%.

#### Razonabilidad:
La razonabilidad mide que los valores de los datos tengan un tipo y tamaño adecuados, es decir, que sean lógicos y comprensibles según lo que se espera para cada atributo. También impone restricciones sobre el tamaño de los valores para evitar errores y asegurar que la base de datos esté bien organizada. En nuestro caso, la razonabilidad es total porque los valores cumplen con lo esperado dentro de su contexto.


#### Identificabilidad:
La identificabilidad calcula el grado en que los registros de datos son identificables de forma única y no son duplicados entre sí.

In [ ]:
# Porcentaje de valores identificables de forma única
# ==============================================================================
(df_meets.nunique().mul(100) / len(df_meets)).sort_values().round(2)

,0
MeetCountry,0.53
Federation,0.71
MeetState,0.94
MeetTown,18.14
Date,31.27
MeetName,61.51
MeetID,100.00
MeetPath,100.00


### Métricas 'openpowerlifting.csv':
De nuevo, antes de adentrarnos en el análisis de las métricas, dedicaremos un momento a revisar el archivo 'openpowerlifting.csv' y comprender la estructura de sus datos.

In [ ]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
df_open.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386414 entries, 0 to 386413
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   MeetID          386414 non-null  int64  
 1   Name            386414 non-null  object 
 2   Sex             386414 non-null  object 
 3   Equipment       386414 non-null  object 
 4   Age             147147 non-null  float64
 5   Division        370571 non-null  object 
 6   BodyweightKg    384012 non-null  float64
 7   WeightClassKg   382602 non-null  object 
 8   Squat4Kg        1243 non-null    float64
 9   BestSquatKg     298071 non-null  float64
 10  Bench4Kg        1962 non-null    float64
 11  BestBenchKg     356364 non-null  float64
 12  Deadlift4Kg     2800 non-null    float64
 13  BestDeadliftKg  317847 non-null  float64
 14  TotalKg         363237 non-null  float64
 15  Place           385322 non-null  object 
 16  Wilks           362194 non-null  float64
dtypes: float64

In [ ]:
# Cabezera del archivo
# ==============================================================================
df_open.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,NaN,20.41,NaN,70.31,138.35,1,155.05
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47


In [ ]:
# Número de datos ausentes por columna
# ==============================================================================
df_open.isna().sum().sort_values()

,0
MeetID,0
Name,0
Sex,0
Equipment,0
Place,133
WeightClassKg,226
BodyweightKg,268
TotalKg,4324
Wilks,4549
Division,4750


#### Precisión:
Como ya comentamos, la precisión de los datos se evalúa comparándolos con una fuente confiable. En este caso, verificamos los valores con la página oficial de Open Powerlifting, no con Kaggle. Al hacerlo, vimos que los valores faltantes se deben a que la web no proporciona esa información, no a un error en la extracción de los datos. Por eso, podemos decir que la precisión es completa en todas las columnas y que la fuente es confiable, ya que refleja con exactitud los datos disponibles al momento de la extracción.

Un punto interesante son los valores negativos que aparecen en las columnas de los cuartos levantamientos. Aunque podría parecer que es un error en la extracción, en realidad no lo es. Los valores negativos indican que se intentó un cuarto levantamiento, pero no se logró, mientras que los positivos corresponden a los casos en los que el atleta sí lo logró.

In [ ]:
# Mostrar los primeros valores un cuarto levantamiento donde no hay datos nulos
# ==============================================================================
df_open[df_open['Squat4Kg'].notnull()]['Squat4Kg'].head()

,Squat4Kg
5,-183.70
20,-183.70
40,367.41
46,195.04
48,-219.99


#### Linaje:
Al igual que con el primer archivo, el linaje de los datos es sólido, ya que provienen directamente de la página oficial de Open Powerlifting, que tiene acceso a la información actualizada de los atletas y competiciones.

#### Semántica:
Para analizar la semántica en el segundo archivo, es importante verificar que los valores en las columnas numéricas sean positivos, ya que se refieren a datos de levantamientos, posiciones o métricas, los cuales deben ser necesariamente positivos.

In [ ]:
# Lista de columnas numéricas del DataFrame
numeric_columns = [
    'Age', 'BodyweightKg', 'BestSquatKg', 'BestBenchKg', 'BestDeadliftKg',
    'TotalKg', 'Wilks', 'Squat4Kg', 'Bench4Kg', 'Deadlift4Kg'
]

# Calcular el porcentaje de valores positivos en cada columna
positive_percentages = {}
for col in numeric_columns:
    positive_count = (df_open[col] > 0).sum()
    total_count = df_open[col].notnull().sum()
    positive_percentage = (positive_count / total_count) * 100 if total_count > 0 else 0
    positive_percentages[col] = positive_percentage

# Convertir el resultado a un DataFrame
positive_percent_df = pd.DataFrame(list(positive_percentages.items()), columns=['Column', 'Positive Percentage'])

# Redondear los porcentajes a 2 decimales
positive_percent_df['Positive Percentage'] = positive_percent_df['Positive Percentage'].round(2)

# Mostrar el DataFrame con los resultados
positive_percent_df




,Column,Positive Percentage
0,Age,100.00
1,BodyweightKg,100.00
2,BestSquatKg,99.99
3,BestBenchKg,99.99
4,BestDeadliftKg,99.99
5,TotalKg,100.00
6,Wilks,100.00
7,Squat4Kg,66.12
8,Bench4Kg,45.02
9,Deadlift4Kg,66.09


#### Estructura:
Al igual que en el caso anterior, el análisis estructural se enfoca en comprobar que los datos sigan un formato y patrón adecuado. En este caso, las columnas de tipo texto son válidas si no contienen caracteres numéricos, mientras que las columnas numéricas son correctas siempre que sus valores sean positivos.

In [ ]:
# Verificar si los valores de la columna 'Name' contienen números
df_open['Name_is_valid'] = df_open['Name'].apply(lambda x: not bool(re.search(r'\d', x)) if isinstance(x, str) else False)

# Filtrar los nombres inválidos
invalid_names = df_open[~df_open['Name_is_valid']]

# Calcular el porcentaje de nombres inválidos
percentage_invalid = (len(invalid_names) / len(df_open['Name'])) * 100

# Mostrar el resultado
print(f"Porcentaje de nombres inválidos: {percentage_invalid:.2f}%")

Porcentaje de nombres inválidos: 0.02%


#### Completitud:
La completitud define el grado en que se rellenan los valores de datos necesarios y no se dejan en blanco.

In [ ]:
# Porcentaje de datos no ausentes por columna
# ==============================================================================
(100 - (df_open.isna().sum() * 100 / len(df_open))).sort_values().round(2)

,0
Squat4Kg,0.51
Bench4Kg,0.87
Deadlift4Kg,1.00
Age,26.50
BestSquatKg,71.60
BestDeadliftKg,76.34
BestBenchKg,94.97
Division,95.63
Wilks,95.81
TotalKg,96.02


#### Consistencia:
En el segundo caso, los registros están todos en un solo archivo, por lo que no hay otra fuente con la que compararlos. La única columna que podríamos revisar es el MeetID, pero dado que solo contiene valores numéricos, podemos decir que la consistencia de los datos es total.

#### Puntualidad:
Al igual que con el primer archivo, no hemos tenido ningún problema al acceder o procesar los datos de este segundo archivo. Por lo tanto, podemos afirmar que la puntualidad es del 100%.

In [ ]:
(df_open.nunique().mul(100) / len(df_open)).sort_values().round(2)

,0
Sex,0.00
Equipment,0.00
WeightClassKg,0.04
Place,0.05
Age,0.14
Squat4Kg,0.27
Bench4Kg,0.30
Deadlift4Kg,0.35
BestDeadliftKg,0.60
BestBenchKg,0.61


In [ ]:
# Número de datos ausentes por columna
# ==============================================================================
df_open.isna().sum().sort_values()

,0
MeetID,0
Name,0
Sex,0
Equipment,0
Place,133
WeightClassKg,226
BodyweightKg,268
TotalKg,4324
Wilks,4549
Division,4750


In [ ]:
# Porcentaje de datos ausentes por columna
# ==============================================================================
(df_open.isna().sum() * 100 / len(df_open)).sort_values().round(2)

,0
MeetID,0.00
Name,0.00
Sex,0.00
Equipment,0.00
Place,0.05
WeightClassKg,0.25
BodyweightKg,0.36
Division,1.71
TotalKg,1.99
Wilks,2.27


In [ ]:
len(df_open['MeetID'].unique())

8482

# **Transformación**

# 1. - Dataset de las competiciones (df_meets)

In [79]:
# Análisis preliminar
# =============================
print('Informaicón df_meets')
print('Dimensiones:')
print(df_meets.shape)
print('=========='*2)
print('N_Unicos:')
print(df_meets.nunique())
print('=========='*2)
print('Nulos')
print(df_meets.isnull().sum())

Informaicón df_meets
Dimensiones:
(8482, 8)
N_Unicos:
MeetID         8482
MeetPath       8482
Federation       60
Date           2652
MeetCountry      45
MeetState        80
MeetTown       1539
MeetName       5217
dtype: int64
Nulos
MeetID            0
MeetPath          0
Federation        0
Date              0
MeetCountry       0
MeetState      2986
MeetTown       1509
MeetName          0
dtype: int64


# Duplicados

Al haber 8482 filas y 8482 instancias de 'MeetID', sabemos que no puede haber duplicados

# Nulos

In [80]:
# Tratamiento faltantes 'MeetState' y 'MeetTown'
# ==============================================================================

# Encontramos valores faltantes en 'MeetTown' y 'MeetState'. Habiendo más en este segundo,
# pero si sabemos la ciudad, sabemos el Estado.

df_meets_state_nulos = df_meets[df_meets['MeetState'].isnull()]
# Vemos los casos en los que disponemos de la ciudad
lista_ciudades_con_estado_null = df_meets_state_nulos[df_meets_state_nulos['MeetTown'].notnull()]['MeetTown'].unique().tolist()

# Revisamos en que casos disponemos de una instacia de MeetTown con su respectivo Estado
diccionario_ciudades_estado = (df_meets[df_meets['MeetTown'].isin(lista_ciudades_con_estado_null)]
                               .loc[lambda x: x['MeetState'].notnull()]
                               .groupby('MeetTown')['MeetState'].first().to_dict())

# Mapear y rellenar los valores faltantes de 'MeetState' en el dataset original
df_meets.loc[
    (df_meets['MeetState'].isnull()) & (df_meets['MeetTown'].notnull()),
    'MeetState'
] = df_meets['MeetTown'].map(diccionario_ciudades_estado)

## Hemos rellenado en base a las relacionese diponibles en el dataset entre 'MeetTown' y 'MeetState'
## Solo nos ha servido para 29 casos. El resto lo vamos a rellenar como 'Desconocido'

# Rellenar valores nulos en MeetState y MeetTown
df_meets['MeetState'] = df_meets['MeetState'].fillna('Desconocido')
df_meets['MeetTown'] = df_meets['MeetTown'].fillna('Desconocido')

# 2. - Dataset de los competidores (df_open)

# Consideraciones iniciales

In [81]:
# 1. Eliminar los cuartos levantamientos
# ==============================================================================
df_open = df_open.drop(['Bench4Kg', 'Deadlift4Kg', 'Squat4Kg'], axis=1)

# 2. Seleccionamos solo aquellos atletas que hayan realizado los 3 moviminetos
# ==============================================================================

df_open = df_open[(df_open['BestBenchKg'].notnull()) & (df_open['BestSquatKg'].notnull()) & (df_open['BestDeadliftKg'].notnull())]

# 3. Cambiamos los levantamientos negativos (fallidos) por 95% del peso
# ==============================================================================

df_open.loc[df_open['BestBenchKg'] < 0, 'BestBenchKg'] = (
    df_open.loc[df_open['BestBenchKg'] < 0, 'BestBenchKg'].abs() * 0.95
)

df_open.loc[df_open['BestDeadliftKg'] < 0, 'BestDeadliftKg'] = (
    df_open.loc[df_open['BestDeadliftKg'] < 0, 'BestDeadliftKg'].abs() * 0.95
)

df_open.loc[df_open['BestSquatKg'] < 0, 'BestSquatKg'] = (
    df_open.loc[df_open['BestSquatKg'] < 0, 'BestSquatKg'].abs() * 0.95
)

# 4. Eliminamos duplicados (158 filas)
# ==============================================================================
df_open = df_open.drop_duplicates()


In [82]:
# Análisis preliminar
# =============================
print(df_open.shape)
print('============'*2)
print('N_Unicos:')
print(df_open.nunique())
print('============'*2)
print('Nulos')
print(df_open.isnull().sum())

(287684, 14)
N_Unicos:
MeetID              7349
Name              113397
Sex                    2
Equipment              4
Age                  160
Division            3274
BodyweightKg        8444
WeightClassKg         48
BestSquatKg         1403
BestBenchKg         1154
BestDeadliftKg      1248
TotalKg             3803
Place                 80
Wilks              48978
dtype: int64
Nulos
MeetID                 0
Name                   0
Sex                    0
Equipment              0
Age               179727
Division           13647
BodyweightKg         600
WeightClassKg       1809
BestSquatKg            0
BestBenchKg            0
BestDeadliftKg         0
TotalKg             1643
Place                735
Wilks               2237
dtype: int64


# Nulos

In [83]:
### 1. 'TotalKg'
# =====================================================================================================================================================
df_open.loc[df_open['TotalKg'].isnull(), 'TotalKg'] = (
    df_open['BestBenchKg'] + df_open['BestDeadliftKg'] + df_open['BestSquatKg']
)

### 2. - 'Division'
# =====================================================================================================================================================
# Cada atleta puede participar en distintas divisiones, hay casos en los que los mismo levantamientos sirven para 2 divisiones distintas, por
# ejemplo, en una división con rango de edad de 40 a 44 años, los levantamientos del atleta podrían entrar aquí y en el open en el que no hay límite.
# En base a esto no podemos determinar de forma certera a que división pertenece por lo que lo más adecuado es rellenar con 'Desconocido'
# Estamos rellenando 13.647 instancias con este valor
df_open['Division'] = df_open['Division'].fillna('Desconocido')

### 3. 'BodyweightKg'
# =====================================================================================================================================================
# Para hacerlo vamos a calcular la relación media entre 'TotalKg' y 'BodyweightKg' en función del sexo
media_total_peso_M = df_open[df_open['Sex'] == 'M']['TotalKg'].div(df_open['BodyweightKg']).mean()
media_total_peso_F = df_open[df_open['Sex'] == 'F']['TotalKg'].div(df_open['BodyweightKg']).mean()

df_open.loc[(df_open['Sex'] == 'F') & (df_open['BodyweightKg'].isnull()), 'BodyweightKg'] = round(df_open['TotalKg'] / media_total_peso_F, 2)
df_open.loc[(df_open['Sex'] == 'M') & (df_open['BodyweightKg'].isnull()), 'BodyweightKg'] = round(df_open['TotalKg'] / media_total_peso_M, 2)



### 4. 'WeightClassKg'.
# =====================================================================================================================================================
# Cuando no se especifique la categoría, vamos a aplicar 'Desconocido', podríamos pensar en asociar la categoría en función del peso del atleta pero
# no es algo que podamos asegurar ya que hay categorías que tienen peso ma´ximo, y otras peso mínimo. De todas formas,
#como tenemos el 'BodyweightKg' en la mayoría de casos, podemos darle más importancia a este que nos está dando una información muy similar
df_open['WeightClassKg'] = df_open['WeightClassKg'].fillna('Desconocido')


### 5. 'Place'.
# =====================================================================================================================================================
# Esto es una variable que no es demasiado relevante para nuestro análisis. Estos valores faltantes se están refiriendo a competiciones en las que no añaden la
# posición fuera de los 3 primeros, pero esto es solo en algunas competiciones. En otros casos como 'MeetID' == 7525, en la categoría 125+, no se ha dado posición a ninguno
# de los 2 participantes. Además de todo esto, nuestro análisis se centra en los mejores casos y en el rendimiento. Por lo que hemos decidido sustituir por: 'Indeterminado'
df_open['Place'] = df_open['Place'].fillna('Indeterminado')


### 6. 'Wilks', es un valor que se calcula a través de distintas variables: 'BodyweightKg', 'Sex', 'TotalKg' y con una serie de coeficientes.
# =====================================================================================================================================================
# A esto hay que añadirle que dependiendo de la competición y del equipamiento, los coeficientes varían. Esto hace que sea imposible estimar de forma
# correcta su valor, sin comprometer fallos relevantes en otras instancias. Por ello lo que creemos que es mejor es sustiuir los valores faltantes por: (-1).
# Así si quisieramos realizar cualquier operación con esta variable como una media, solo tendríamos que omitir los casos en los que 'Wilks' == -1.
df_open['Wilks'] = df_open['Wilks'].fillna(-1)

### 7. 'Age'
# =====================================================================================================================================================
# Debido a la gran cantidad de nulos que encontramos, cerca de un 70% no podemos imputar los valores, además dependiendo de la competición
# hay unos rangos de edad. Hemos decidido eliminar la columna, aunque si estuviera rellenada sería una columna muy útil.
df_open = df_open.drop(columns=['Age'])

# 3. Creación y modificiaciones de columnas (Ids también)

In [84]:
#################### df_meets ####################

### Creamos columnas para la dimensión tiempo
# =====================================================================================================================================================
# Convertir la columna 'Date' a formato datetime
df_meets['Date'] = pd.to_datetime(df_meets['Date'])

# Extraer el año
df_meets['Year'] = df_meets['Date'].dt.year

# Calcular el trimestre de la fecha
df_meets['Quarter'] = df_meets['Date'].dt.quarter

# Extraer el mes
df_meets['Month'] = df_meets['Date'].dt.month

# Extraer el día
df_meets['Day'] = df_meets['Date'].dt.day

# Extraer el día de la semana numérico (lunes=1, domingo=7)
df_meets['DayWeek'] = df_meets['Date'].dt.weekday + 1

# Añadir una columna para indicar si es fin de semana (sábado = 6, domingo = 7)
df_meets['IsWeekend'] = df_meets['DayWeek'].isin([6, 7])
# Convertir valores a 'Yes' o 'No'
df_meets['IsWeekend'] = df_meets['IsWeekend'].replace({True: 'Yes', False: 'No'})


### Generación de IDs
# =====================================================================================================================================================
# idUbication
df_meets['idUbication'] = df_meets[['MeetCountry', 'MeetState', 'MeetTown']].astype(str).agg('-'.join, axis=1).factorize()[0] + 1

# Generamos el id único (lo hacemos ordenando por fechas más antiguas)
df_meets['idDate'] = df_meets['Date'].rank(method='dense').astype(int)

# MeetID --> idMeet
df_meets.rename(columns={'MeetID': 'idMeet'}, inplace=True)

In [85]:
#################### df_open ####################

### Creamos distintas métricas relevantes para nuestro análisis
# =====================================================================================================================================================
# Métrica 1. - Relación entre 'BestSquatKg' y 'Bench4Kg'
df_open['SquatBenchRatio'] = round(df_open['BestSquatKg'] / df_open['BestBenchKg'], 3)

# Métrica 2. - Impacto de la sentidilla en el total de Kilos movidos
df_open['SquatImpact'] = round(df_open['BestSquatKg'] / df_open['TotalKg'], 3)

# Métrica 3. - Relación entre los kilos movidos y el peso del competidor
df_open['KgPerWeight'] = round(df_open['TotalKg'] / df_open['BodyweightKg'], 3)


### Generación de IDs
# =====================================================================================================================================================
# idAthlete
df_open['idAthlete'] = df_open['Name'].factorize()[0] + 1

# idEquip
df_open['idEquip'] = df_open['Equipment'].factorize()[0] + 1

# idWeight
df_open['idWeight'] = df_open[['Name', 'BodyweightKg', 'WeightClassKg', 'Division']].astype(str).agg('-'.join, axis=1).factorize()[0] + 1

# idPerformace
df_open['idPerformance'] = range(len(df_open))

# MeetID --> idMeet
df_open.rename(columns={'MeetID': 'idMeet'}, inplace=True)


# 4. Creación de tablas y csv

In [86]:
### 1. Crear DataFrames para cada dimensión
# =====================================================================================================================================================
Dimension_Weight = df_open[['idWeight', 'Division', 'BodyweightKg', 'WeightClassKg']].drop_duplicates(subset='idWeight')
Dimension_Athlete = df_open[['idAthlete', 'idWeight', 'Name', 'Sex']].drop_duplicates(subset='idAthlete')
Dimension_Time = df_meets[['idDate', 'Date', 'Year', 'Quarter', 'Month', 'Day', 'DayWeek', 'IsWeekend']].drop_duplicates(subset='idDate')
Dimension_Ubication = df_meets[['idUbication', 'MeetCountry', 'MeetState', 'MeetTown']].drop_duplicates(subset='idUbication')
Dimension_Meet = df_meets[['idMeet', 'MeetName', 'Federation', 'MeetPath']].drop_duplicates(subset='idMeet')
Dimension_Performance = df_open[['idPerformance', 'Equipment', 'BestSquatKg', 'BestBenchKg', 'BestDeadliftKg', 'TotalKg', 'Place', 'Wilks']].drop_duplicates(subset='idPerformance')

Facts_Competition = df_open.merge(
    df_meets[['idMeet', 'idUbication', 'idDate']],
    on='idMeet',
    how='inner'
)
# Seleccionar las columnas finales
Facts_Competition = Facts_Competition[[
    'idMeet', 'idPerformance', 'idDate', 'idUbication', 'idAthlete', 'idWeight',
    'SquatBenchRatio', 'SquatImpact', 'KgPerWeight'
]]

In [87]:
### 2. Guardar las dimensiones en archivos CSV
# =====================================================================================================================================================
Dimension_Weight.to_csv('Dimension_Weight.csv', index=False)
Dimension_Athlete.to_csv('Dimension_Athlete.csv', index=False)
Dimension_Time.to_csv('Dimension_Time.csv', index=False)
Dimension_Ubication.to_csv('Dimension_Ubication.csv', index=False)
Dimension_Meet.to_csv('Dimension_Meet.csv', index=False)
Dimension_Performance.to_csv('Dimension_Performance.csv', index=False)

# Guardar la tabla de hechos en un archivo CSV
Facts_Competition.to_csv('Facts_Competition.csv', index=False)

# Consultas finales

In [51]:
# El levantador que más peso ha movido mueve un 13% más en sentadilla en relación con el total que la media
df_open[df_open['TotalKg'] == df_open['TotalKg'].max()]['SquatImpact'] / df_open['SquatImpact'].mean()

,SquatImpact
57011,1.133941


In [88]:
# Con esto podemos ver que en el caso de los hombres no existe tanta diferencia entre
df_open.groupby('Sex')['SquatBenchRatio'].mean()

,SquatBenchRatio
Sex,
F,1.794583
M,1.549646


In [89]:
# Los atletas que son capaces de mover más peso con relación al suyo son aquellos de categorías de unos 60kg
max_kg_per_weight = df_open.groupby('Sex')['KgPerWeight'].max()
df_open[df_open.apply(lambda row: row['KgPerWeight'] == max_kg_per_weight[row['Sex']], axis=1)]

,idMeet,Name,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,SquatBenchRatio,SquatImpact,KgPerWeight,idAthlete,idEquip,idWeight,idPerformance
53119,1327,Heidi Howar,F,Multi-ply,Open,59.24,60,263.08,179.17,235.87,678.12,1,763.55,1.468,0.388,11.447,17827,4,38387,39902
164412,5529,Lones Green,M,Multi-ply,Juniors,63.19,140,362.87,208.65,292.57,864.09,1,703.82,1.739,0.420,13.674,41905,4,111317,115377


In [92]:
# Categorías más competitivas
df_open.groupby('WeightClassKg').agg(
    TotalKg_mean=('TotalKg', 'mean'),
    KgPerWeight_mean=('KgPerWeight', 'mean')
).round(3).sort_values(by='KgPerWeight_mean', ascending=False)


,TotalKg_mean,KgPerWeight_mean
WeightClassKg,,
80,592.119,7.608
95,688.238,7.469
66,450.222,6.968
59,401.217,6.968
74,488.925,6.766
83,525.489,6.506
53,326.284,6.363
93,561.947,6.234
145+,904.722,6.085


In [93]:
# Peso máximo movido según el equipamiento
max_total_kg = df_open.groupby('Equipment')['TotalKg'].max()

# Filtra las filas que tienen esos valores máximos
df_open[df_open.apply(lambda row: row['TotalKg'] == max_total_kg[row['Equipment']], axis=1)]


,idMeet,Name,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,SquatBenchRatio,SquatImpact,KgPerWeight,idAthlete,idEquip,idWeight,idPerformance
57011,1412,Dave Hoff,M,Multi-ply,Pro Open,131.09,140,560.19,437.72,367.41,1365.31,1,771.07,1.280,0.410,10.415,1862,4,41030,42563
129736,4875,Andrey Malanichev,M,Wraps,Desconocido,187.34,140+,485.00,255.00,400.00,1140.00,1,-1.00,1.902,0.425,6.085,3898,1,87968,91597
298069,7267,Ray Williams,M,Raw,R-O,182.26,120+,477.50,235.00,392.50,1105.00,1,593.51,2.032,0.432,6.063,22418,3,219347,227324
298158,7271,Blaine Sumner,M,Single-ply,M-O,170.68,120+,505.00,410.00,357.50,1272.50,1,690.71,1.232,0.397,7.455,22102,2,219356,227333


In [63]:
# Como influye a nivel medio el uso de equipamiento
df_open.groupby('Equipment')[['BestSquatKg', 'BestBenchKg', 'BestDeadliftKg']].max()

,BestSquatKg,BestBenchKg,BestDeadliftKg
Equipment,,,
Multi-ply,573.79,455.86,410.5
Raw,477.50,320.00,420.5
Single-ply,505.00,410.00,420.0
Wraps,500.00,290.30,440.0
